In [ ]:
#install the requirements
!pip install -r requirements.txt

In [ ]:
import sys
sys.path.append('src')
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch
from utils import visualize_tensors,inference,plot_tensor,inference_mix_prec
from support_image_import import *
from collections import defaultdict
from prune_model import prune_model
import datetime
import warnings
warnings.filterwarnings('ignore')
torch.manual_seed(156)
np.random.seed(156)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
import sys
import torch.nn as nn 
sys.path.append('UniverSeg')


from universeg import universeg
model = universeg(pretrained=True)
model = model.to(device).eval()


In [ ]:
size = (256,256)
support_size = 64
#modify the images folder
support_images_folder = "data/all2/images"
support_masks_folder = "data/all2/mask"
image_path = "data/test/images/ISIC_0000137.jpg"
test_images = "data/test/images"
test_masks = "data/test/mask"

In [ ]:
#structural_comparison original algorith with gpu
sp_im,sp_ms,img_test = import_support_image_structural(support_size,support_images_folder,support_masks_folder,image_path,device,size)

In [ ]:
#dense_vector_comparison
sp_im,sp_ms,img_test = import_support_image_dense_vector(support_size,support_images_folder,support_masks_folder,image_path,device,size)

In [ ]:
#hash comparison
sp_im,sp_ms,img_test = import_support_image_hash(support_size,support_images_folder,support_masks_folder,image_path,device,size)

In [ ]:
#hash with augmentation
sp_im,sp_ms,img_test = import_support_image_hash_aug(support_size,support_images_folder,support_masks_folder,image_path,device,size)

In [ ]:
#lpips comparison
sp_im,sp_ms,img_test = import_support_image_lpips(support_size,support_images_folder,support_masks_folder,image_path,device,size)

In [ ]:
test_img,test_mask,_ = import_support_image_lpips(8,test_images,test_masks,image_path,device,size)

In [ ]:
#visualize the support images
plot_tensor(test_img)

In [ ]:
#prune model test
prune_model(model,0.25)

In [ ]:
#compile the model increase the support image size but loss some speed
model = torch.compile(model)

In [ ]:
num_support = 36
# run inference
arrtest1 = sp_im[:num_support].detach()
arrtest2 = sp_ms[:num_support].detach()
with torch.inference_mode():
    logits = model(img_test[None].to(device), arrtest1[None].to(device), arrtest2[None].to(device))[0].to('cpu').float()
pred = torch.sigmoid(logits)
# visualize
res = {'data': [img_test, pred, pred > 0.5]}
titles = col_names=['image', 'pred (soft)', 'pred (hard)']
visualize_tensors(res, col_wrap=3, col_names=titles)
del logits,pred,res
torch.cuda.empty_cache()

In [ ]:
#run multiple predictions
n_predictions = 6
num_support = 32
idxs = np.random.permutation(len(test_img)-1)[:n_predictions]
time = []
# run inference
arrtest1 = sp_im[:num_support]
arrtest2 = sp_ms[:num_support]
results = defaultdict(list)
x = datetime.datetime.now()
for i in tqdm(idxs):
    x = datetime.datetime.now()
    image, label = test_img[i],test_mask[i]
    vals = inference(model, image, label, arrtest1, arrtest2,device)
    for k, v in vals.items():
        results[k].append(v)
    del image,label,vals
    torch.cuda.empty_cache()
    y = datetime.datetime.now()
    time.append(y-x)
scores = results.pop('score')
visualize_tensors(results, col_names=[f'Dice = {100*s:.1f}' for s in scores], title='Test Predictions - Different Label', col_wrap=n_predictions)


In [ ]:
#return time and inference score mean
scores1 = np.array(scores)
scores1 = np.nan_to_num(scores).mean() 
print("mean dice score:",scores1)
sum = datetime.datetime.now() - datetime.datetime.now()
for i in range(n_predictions):
    sum += time[i]
print("mean time per inference:",sum / n_predictions)

In [ ]:
#run multiple predictions on mixed presition

n_predictions = 6
num_support = 32
idxs = np.random.permutation(len(test_img)-1)[:n_predictions]
time = []
# run inference
arrtest1 = sp_im[:num_support]
arrtest2 = sp_ms[:num_support]
results = defaultdict(list)
x = datetime.datetime.now()
for i in tqdm(idxs):
    x = datetime.datetime.now()
    image, label = test_img[i],test_mask[i]
    vals = inference_mix_prec(model, image, label, arrtest1, arrtest2,device)
    for k, v in vals.items():
        results[k].append(v)
    del image,label,vals
    torch.cuda.empty_cache()
    y = datetime.datetime.now()
    time.append(y-x)
scores = results.pop('score')
visualize_tensors(results, col_names=[f'Dice = {100*s:.1f}' for s in scores], title='Test Predictions - Different Label', col_wrap=n_predictions)

In [ ]:
#return time and inference score mean
scores1 = np.array(scores)
scores1 = np.nan_to_num(scores).mean() 
print("mean dice score:",scores1)
sum = datetime.datetime.now() - datetime.datetime.now()
for i in range(n_predictions):
    sum += time[i]
print("mean time per inference:",sum / n_predictions)

In [ ]:
#prediction on different support size
import pandas as pd
n_predictions = 2
support_set_sizes = [1, 2, 4, 8,16,32,64]

scores = []
scores1 = []
for i in idxs:
    results = defaultdict(list)
    for N in tqdm(support_set_sizes):
        image, label = test_img[i],test_mask[i]
        vals = inference(model, image, label, sp_im[:N], sp_ms[:N],device)
        for k, v in vals.items():
            results[k].append(v)
        del image,label,vals
        torch.cuda.empty_cache()
    scores.append(results['score'])

    #uncomment the next lines to plot the predictions

    #scores = results.pop('score')
    #col_names = [f'N = {N}' for N in support_set_sizes]
    #col_names = [col+f'\nDice: {100*score:.1f}' for col, score in zip(col_names, scores)]
    #visualize_tensors(results, col_names=col_names, title='Test Predictions for varying Support Set size $N$', col_wrap=len(support_set_sizes))

In [ ]:
#plot basic stats to compare the results of different support sizes
df = pd.DataFrame(scores)
df.columns = support_set_sizes
df.describe()

In [ ]:
#plot the results
scores = results.pop('score')
col_names = [f'N = {N}' for N in support_set_sizes]
col_names = [col+f'\nDice: {100*score:.1f}' for col, score in zip(col_names, scores)]
visualize_tensors(results, col_names=col_names, title='Test Predictions for varying Support Set size $N$', col_wrap=len(support_set_sizes))